In [106]:
import torch
import torch.nn as nn

In [107]:
def loss_repeated_labels(roundedOutput):
  used_labels, counts = torch.unique(roundedOutput, return_counts=True)
  counts = counts.type(torch.DoubleTensor)
  print(counts)
  print(roundedOutput.shape)
  print(counts.shape)
  return torch.var(counts, unbiased=False) + (roundedOutput.shape[0] - counts.shape[0])**2

In [108]:
y_pred = torch.tensor([
  [0., 1., 1., 2., 2., 3., 1.],
  [0., 1., 1., 2., 2., 3., 1.],
  [0., 2., 3., 3., 1., 4., 5.],
  [0., 2., 2., 3., 1., 4., 5.],
])

y_true = torch.tensor([
  [0., 1., 2., 3., 4., 5., 6.],
  [6., 5., 4., 3., 2., 1., 0.],
  [1., 0., 2., 4., 6., 3., 5.],
  [1., 2., 0., 3., 4., 6., 5.],
])

y_pred.requires_grad = True

print(y_pred)
print(y_pred.dtype)

loss = loss_repeated_labels(y_pred)

tensor([[0., 1., 1., 2., 2., 3., 1.],
        [0., 1., 1., 2., 2., 3., 1.],
        [0., 2., 3., 3., 1., 4., 5.],
        [0., 2., 2., 3., 1., 4., 5.]], requires_grad=True)
torch.float32
tensor([4., 8., 7., 5., 2., 2.], dtype=torch.float64)
torch.Size([4, 7])
torch.Size([6])


In [109]:
loss = loss_repeated_labels(y_true)

tensor([4., 4., 4., 4., 4., 4., 4.], dtype=torch.float64)
torch.Size([4, 7])
torch.Size([7])


In [110]:
mse = nn.MSELoss()

def new_loss_repeated_labels(roundedOutput, use_var):
  batch_size = roundedOutput.shape[0]

  used_labels, counts = torch.unique(roundedOutput, return_counts=True)
  counts = counts.type(torch.FloatTensor)

  counts_shape = counts.shape[0]
  print(counts)

  optimalCounts = torch.ones(counts_shape) * batch_size
  print(optimalCounts)

  if use_var:
    var_loss = torch.var(counts, unbiased=False)
    return var_loss
  else:
    mse_loss = mse(counts, optimalCounts)
    return mse_loss

use_var = new_loss_repeated_labels(y_pred, use_var=True)
use_mse = new_loss_repeated_labels(y_pred, use_var=False)

mse_loss = ((y_pred - y_true)**2).mean()
mse_loss2 = mse(y_pred, y_true)
print((use_mse + mse_loss).backward())
print((use_var + mse_loss2).backward())

tensor([4., 8., 7., 5., 2., 2.])
tensor([4., 4., 4., 4., 4., 4.])
tensor([4., 8., 7., 5., 2., 2.])
tensor([4., 4., 4., 4., 4., 4.])
None
None


In [111]:
use_var = new_loss_repeated_labels(y_true, use_var=True)
use_mse = new_loss_repeated_labels(y_true, use_var=False)

tensor([4., 4., 4., 4., 4., 4., 4.])
tensor([4., 4., 4., 4., 4., 4., 4.])
tensor([4., 4., 4., 4., 4., 4., 4.])
tensor([4., 4., 4., 4., 4., 4., 4.])
